In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import atomap.api as am
import hyperspy.api as hs

In [2]:
#!pip install ipython --upgrade
#!pip install matplotlib --upgrade

In [2]:
s = hs.load('9_registered_1pm.hspy') 

In [3]:
s.plot()
#s.axes_manager #pixel size = 0.02622468769550323nm

In [32]:
s_temp = s.inav[0].copy()
s_temp = s.sum(0)
s_temp.plot()

In [42]:
s_temp.isig[10:120,320:].plot()

In [43]:
s_temp

<Signal2D, title: 9Frame Grabber 60 frames, dimensions: (|479, 473)>

In [44]:
s_crop = s_temp.isig[10:120,320:]

In [45]:
s_peaks = am.get_feature_separation(s_crop, separation_range=(1, 10), show_progressbar=False)

In [46]:
s_peaks.plot()

In [47]:
atom_positions = am.get_atom_positions(s_crop, 5)

In [67]:
atom_positions_A = am.add_atoms_with_gui(s_crop, atom_positions_A_copy)

In [59]:
atom_positions_A_copy = atom_positions_A.copy()

In [68]:
sublattice_In = am.Sublattice(atom_positions_A, image=s_crop.data)
sublattice_In

<Sublattice,  (atoms:74,planes:0)>

In [50]:
s_crop.axes_manager
x_offset = s_crop.axes_manager[0].offset
y_offset = s_crop.axes_manager[1].offset
x_scale = s_crop.axes_manager[0].scale
y_scale = s_crop.axes_manager[1].scale

In [69]:
sublattice_In.find_nearest_neighbors(nearest_neighbors=20)
sublattice_In.refine_atom_positions_using_center_of_mass(percent_to_nn=0.2)
sublattice_In.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)

Center of mass:   0%|          | 0/74 [00:00<?, ?it/s]

Gaussian fitting:   0%|          | 0/74 [00:00<?, ?it/s]

In [77]:
sublattice_In.get_position_history().plot()

In [71]:
sublattice_In.construct_zone_axes(atom_plane_tolerance=2)
sublattice_In

<Sublattice,  (atoms:74,planes:14)>

In [72]:
sublattice_In.plot_planes()

In [73]:
zone_axis_001 = sublattice_In.zones_axis_average_distances[2]
B_positions = sublattice_In.find_missing_atoms_from_zone_vector(zone_axis_001)

In [75]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_without_A = remove_atoms_from_image_using_2d_gaussian(sublattice_In.image, sublattice_In)

Subtracting atoms:   0%|          | 0/74 [00:00<?, ?it/s]

In [427]:
B_positions=am.add_atoms_with_gui(image_without_A, B_positions)

In [453]:
B_positions=am.add_atoms_with_gui(sublattice_In.image, B_positions)

In [450]:
B_positions_copy = B_positions.copy()

In [454]:
sublattice_B = am.Sublattice(B_positions, image_without_A, color='blue')
sublattice_B.find_nearest_neighbors(nearest_neighbors=30)
#sublattice_B.construct_zone_axes()
#sublattice_B.refine_atom_positions_using_center_of_mass(percent_to_nn=0.2)
sublattice_B.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)

Gaussian fitting:   0%|          | 0/103 [00:00<?, ?it/s]

C:\Users\mengk\HyperSpy-bundle\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mengk\HyperSpy-bundle\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


WARNING | Hyperspy | Covariance of the parameters could not be estimated. Estimated parameter standard deviations will be np.nan. (hyperspy.model:1411)
WARNING | Hyperspy | `m.fit()` did not exit successfully. Reason: Number of calls to function has reached maxfev = 1400. (hyperspy.model:1877)
WARNING | Hyperspy | Covariance of the parameters could not be estimated. Estimated parameter standard deviations will be np.nan. (hyperspy.model:1411)
WARNING | Hyperspy | `m.fit()` did not exit successfully. Reason: Number of calls to function has reached maxfev = 1400. (hyperspy.model:1877)


In [430]:
sublattice_B.get_position_history().plot()

In [455]:
atom_lattice = am.Atom_Lattice(image=s_crop.data, name='test', sublattice_list=[sublattice_In, sublattice_B])
atom_lattice.plot()

In [476]:
atom_lattice.save("whole atom_lattice.hdf5", overwrite=True)

In [477]:
import numpy as np
blue_pure_beta_x_ranges = [(74, 82), (87, 93), (100,107)]
red_pure_beta_x_ranges = [(81, 87), (97, 102)]
blue_pure_beta_coord = []
red_pure_beta_coord = []


# Function to slice the positions based on x range and calculate distances
def filter_dots(sublattice_A, sublattice_B, blue_ranges, red_ranges):
    distances = []
    Bx_position = sublattice_B.x_position
    By_position = sublattice_B.y_position
    Rx_position = sublattice_A.x_position
    Ry_position = sublattice_A.y_position
    for blue_range in blue_ranges:
        blue_x = Bx_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1])]
        blue_y = By_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1])]
        blue_pure_beta_coord.append((blue_x,blue_y))
    for red_range in red_ranges:
        red_x = Rx_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1])]
        red_y = Ry_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1])]
        red_pure_beta_coord.append((red_x,red_y))
    return distances

# Call the function and calculate distances
distances = filter_dots(sublattice_In, sublattice_B,blue_pure_beta_x_ranges, red_pure_beta_x_ranges)
blue_pure_beta_column_x = [blue_pure_beta_coord[i][0] for i in range(len(blue_pure_beta_coord))]
blue_pure_beta_column_y = [blue_pure_beta_coord[i][1] for i in range(len(blue_pure_beta_coord))]
red_pure_beta_column_x = [red_pure_beta_coord[i][0] for i in range(len(red_pure_beta_coord))]
red_pure_beta_column_y = [red_pure_beta_coord[i][1] for i in range(len(red_pure_beta_coord))]

In [483]:
import numpy as np
from math import sqrt
def sort_dots(x, y):
    sorted_indices = np.argsort(y)
    sorted_x = x[sorted_indices]
    sorted_y = y[sorted_indices]
    return sorted_x, sorted_y
def distance(x1, x2, y1, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
def plot_center_to_nearest_reds(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >rx and ry < bcy and d <15:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d * x_scale)  # Scale and append the distance
            elif bcx <rx and ry > bcy and d <15:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d* x_scale)  # Scale and append the distance
    return distance_list

plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_pure_beta_column_x )):
    blue_pure_beta_column_x[i],blue_pure_beta_column_y[i]=sort_dots(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i])
    plt.scatter(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i],color='b' )
for i in range(len(red_pure_beta_column_x )):
    red_pure_beta_column_x[i],red_pure_beta_column_y[i]=sort_dots(red_pure_beta_column_x[i],red_pure_beta_column_y[i])
    plt.scatter(red_pure_beta_column_x[i],red_pure_beta_column_y[i],color='r' )
all_red_pure_beta_x = np.concatenate([red_pure_beta_column_x[0], red_pure_beta_column_x[1]])
all_red_pure_beta_y = np.concatenate([red_pure_beta_column_y[0], red_pure_beta_column_y[1]])
all_blue_pure_beta_x = np.concatenate([blue_pure_beta_column_x[0], blue_pure_beta_column_x[1],blue_pure_beta_column_x[2]])
all_blue_pure_beta_y = np.concatenate([blue_pure_beta_column_y[0], blue_pure_beta_column_y[1],blue_pure_beta_column_y[2]])
distance_list_pure_beta = []
distance_list_pure_beta.extend(plot_center_to_nearest_reds(all_blue_pure_beta_x, all_blue_pure_beta_y, all_red_pure_beta_x, all_red_pure_beta_y, 'g-', x_scale))
plt.show()

In [481]:
import numpy as np
blue_def_beta_x_ranges = [(42, 48), (51, 56), (63,70)]
red_def_beta_x_ranges = [(45, 50), (55, 65)]
blue_def_beta_coord = []
red_def_beta_coord = []


# Function to slice the positions based on x range and calculate distances
def filter_dots(blue_coord,red_coord, sublattice_A, sublattice_B, blue_ranges, red_ranges):
    Bx_position = sublattice_B.x_position
    By_position = sublattice_B.y_position
    Rx_position = sublattice_A.x_position
    Ry_position = sublattice_A.y_position
    for blue_range in blue_ranges:
        blue_x = Bx_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1])]
        blue_y = By_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1])]
        blue_def_beta_coord.append((blue_x,blue_y))
    for red_range in red_ranges:
        red_x = Rx_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1])]
        red_y = Ry_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1])]
        red_def_beta_coord.append((red_x,red_y))
    return blue_coord,red_coord, 

# Call the function and calculate distances
blue_def_beta_coord,red_def_beta_coord = filter_dots(blue_def_beta_coord,red_def_beta_coord,sublattice_In, sublattice_B,blue_def_beta_x_ranges, red_def_beta_x_ranges)
blue_def_beta_column_x = [blue_def_beta_coord[i][0] for i in range(len(blue_def_beta_coord))]
blue_def_beta_column_y = [blue_def_beta_coord[i][1] for i in range(len(blue_def_beta_coord))]
red_def_beta_column_x = [red_def_beta_coord[i][0] for i in range(len(red_def_beta_coord))]
red_def_beta_column_y = [red_def_beta_coord[i][1] for i in range(len(red_def_beta_coord))]

In [487]:
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_def_beta_column_x )):
    blue_def_beta_column_x[i],blue_def_beta_column_y[i]=sort_dots(blue_def_beta_column_x[i],blue_def_beta_column_y[i])
    plt.scatter(blue_def_beta_column_x[i],blue_def_beta_column_y[i],color='b' )
for i in range(len(red_def_beta_column_x )):
    red_def_beta_column_x[i],red_def_beta_column_y[i]=sort_dots(red_def_beta_column_x[i],red_def_beta_column_y[i])
    plt.scatter(red_def_beta_column_x[i],red_def_beta_column_y[i],color='r' )
all_red_def_beta_x = np.concatenate([red_def_beta_column_x[0], red_def_beta_column_x[1]])
all_red_def_beta_y = np.concatenate([red_def_beta_column_y[0], red_def_beta_column_y[1]])
all_blue_def_beta_x = np.concatenate([blue_def_beta_column_x[0], blue_def_beta_column_x[1],blue_def_beta_column_x[2]])
all_blue_def_beta_y = np.concatenate([blue_def_beta_column_y[0], blue_def_beta_column_y[1],blue_def_beta_column_y[2]])
distance_list_def_beta = []
distance_list_def_beta.extend(plot_center_to_nearest_reds(all_blue_def_beta_x, all_blue_def_beta_y, all_red_def_beta_x, all_red_def_beta_y, 'g-', x_scale))
plt.show()

In [486]:
import numpy as np
blue_trans_upper_x_ranges = [(2, 8), (15, 23), (23,33)]
red_trans_upper_x_ranges = [(6, 14), (14, 24),(25,38)]
y_trans_upper = 102

blue_trans_upper_coord = []
red_trans_upper_coord = []


# Function to slice the positions based on x range and calculate distances
def filter_dots(blue_coord,red_coord, sublattice_A, sublattice_B, blue_ranges, red_ranges,y):
    Bx_position = sublattice_B.x_position
    By_position = sublattice_B.y_position
    Rx_position = sublattice_A.x_position
    Ry_position = sublattice_A.y_position
    for blue_range in blue_ranges:
        blue_x = Bx_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1]) & (By_position < y) ]
        blue_y = By_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1]) & (By_position < y ) ]
        blue_coord.append((blue_x,blue_y))
    for red_range in red_ranges:
        red_x = Rx_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1]) & (Ry_position < y ) ]
        red_y = Ry_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1]) & (Ry_position < y) ]
        red_coord.append((red_x,red_y))
    return blue_coord,red_coord

# Call the function and calculate distances
blue_trans_upper_coord,red_trans_upper_coord  = filter_dots(blue_trans_upper_coord,red_trans_upper_coord ,sublattice_In, sublattice_B,blue_trans_upper_x_ranges, red_trans_upper_x_ranges,y_trans_upper)
blue_trans_upper_column_x = [blue_trans_upper_coord[i][0] for i in range(len(blue_trans_upper_coord))]
blue_trans_upper_column_y = [blue_trans_upper_coord[i][1] for i in range(len(blue_trans_upper_coord))]
red_trans_upper_column_x = [red_trans_upper_coord[i][0] for i in range(len(red_trans_upper_coord))]
red_trans_upper_column_y = [red_trans_upper_coord[i][1] for i in range(len(red_trans_upper_coord))]

In [489]:
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_trans_upper_column_x )):
    blue_trans_upper_column_x[i],blue_trans_upper_column_y[i]=sort_dots(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i])
    plt.scatter(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i],color='b' )
for i in range(len(red_trans_upper_column_x )):
    red_trans_upper_column_x[i],red_trans_upper_column_y[i]=sort_dots(red_trans_upper_column_x[i],red_trans_upper_column_y[i])
    plt.scatter(red_trans_upper_column_x[i],red_trans_upper_column_y[i],color='r')
def plot_bonds_upper(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if d <7 and d >2:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d * x_scale)  # Scale and append the distance
    return distance_list
all_red_trans_upper_x = np.concatenate([red_trans_upper_column_x[0], red_trans_upper_column_x[1],red_trans_upper_column_x[2]])
all_red_trans_upper_y = np.concatenate([red_trans_upper_column_y[0], red_trans_upper_column_y[1], red_trans_upper_column_y[2]])
all_blue_trans_upper_x = np.concatenate([blue_trans_upper_column_x[0], blue_trans_upper_column_x[1],blue_trans_upper_column_x[2]])
all_blue_trans_upper_y = np.concatenate([blue_trans_upper_column_y[0], blue_trans_upper_column_y[1],blue_trans_upper_column_y[2]])
distance_list__trans_upper = []
distance_list__trans_upper.extend(plot_bonds_upper(all_blue_trans_upper_x, all_blue_trans_upper_y, all_red_trans_upper_x, all_red_trans_upper_y, 'g-', x_scale))
plt.show()

In [465]:
import numpy as np
blue_trans_lower_x_ranges = [(2, 12), (15, 23), (23,33)]
red_trans_lower_x_ranges = [(6, 14), (23,38)]
y_trans_lower = 102

blue_trans_lower_coord = []
red_trans_lower_coord = []


# Function to slice the positions based on x range and calculate distances
def filter_dots(blue_coord,red_coord, sublattice_A, sublattice_B, blue_ranges, red_ranges,y):
    Bx_position = sublattice_B.x_position
    By_position = sublattice_B.y_position
    Rx_position = sublattice_A.x_position
    Ry_position = sublattice_A.y_position
    for blue_range in blue_ranges:
        blue_x = Bx_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1]) & (By_position > y) ]
        blue_y = By_position[(Bx_position >= blue_range[0]) & (Bx_position <= blue_range[1]) & (By_position > y ) ]
        blue_coord.append((blue_x,blue_y))
    for red_range in red_ranges:
        red_x = Rx_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1]) & (Ry_position > y ) ]
        red_y = Ry_position[(Rx_position >= red_range[0]) & (Rx_position <= red_range[1]) & (Ry_position > y) ]
        red_coord.append((red_x,red_y))
    return blue_coord,red_coord

# Call the function and calculate distances
blue_trans_lower_coord,red_trans_lower_coord  = filter_dots(blue_trans_lower_coord,red_trans_lower_coord ,sublattice_In, sublattice_B,blue_trans_lower_x_ranges, red_trans_lower_x_ranges,y_trans_lower)
blue_trans_lower_column_x = [blue_trans_lower_coord[i][0] for i in range(len(blue_trans_lower_coord))]
blue_trans_lower_column_y = [blue_trans_lower_coord[i][1] for i in range(len(blue_trans_lower_coord))]
red_trans_lower_column_x = [red_trans_lower_coord[i][0] for i in range(len(red_trans_lower_coord))]
red_trans_lower_column_y = [red_trans_lower_coord[i][1] for i in range(len(red_trans_lower_coord))]

In [510]:
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_trans_lower_column_x )):
    blue_trans_lower_column_x[i],blue_trans_lower_column_y[i]=sort_dots(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i])
    plt.scatter(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i],color='b' )
for i in range(len(red_trans_lower_column_x )):
    red_trans_lower_column_x[i],red_trans_lower_column_y[i]=sort_dots(red_trans_lower_column_x[i],red_trans_lower_column_y[i])
    plt.scatter(red_trans_lower_column_x[i],red_trans_lower_column_y[i],color='r')
def plot_bonds_lower(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if d <8.7 and d>8.5:
                pass
                #plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                #distance_list.append(d * x_scale)  # Scale and append the distance
            elif d <9:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d * x_scale)  # Scale and append the distance
    return distance_list
all_red_trans_lower_x = np.concatenate([red_trans_lower_column_x[0], red_trans_lower_column_x[1]])
all_red_trans_lower_y = np.concatenate([red_trans_lower_column_y[0], red_trans_lower_column_y[1]])
all_blue_trans_lower_x = np.concatenate([blue_trans_lower_column_x[0], blue_trans_lower_column_x[1],blue_trans_lower_column_x[2]])
all_blue_trans_lower_y = np.concatenate([blue_trans_lower_column_y[0], blue_trans_lower_column_y[1],blue_trans_lower_column_y[2]])
distance_list_trans_lower = []
distance_list_trans_lower.extend(plot_bonds_lower(all_blue_trans_lower_x, all_blue_trans_lower_y, all_red_trans_lower_x, all_red_trans_lower_y, 'g-', x_scale))
plt.show()

In [475]:
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_pure_beta_column_x )):
    blue_pure_beta_column_x[i],blue_pure_beta_column_y[i]=sort_dots(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i])
    plt.scatter(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i],color='b' )
for i in range(len(red_pure_beta_column_x )):
    red_pure_beta_column_x[i],red_pure_beta_column_y[i]=sort_dots(red_pure_beta_column_x[i],red_pure_beta_column_y[i])
    plt.scatter(red_pure_beta_column_x[i],red_pure_beta_column_y[i],color='r' )

for i in range(len(blue_def_beta_column_x )):
    blue_def_beta_column_x[i],blue_def_beta_column_y[i]=sort_dots(blue_def_beta_column_x[i],blue_def_beta_column_y[i])
    plt.scatter(blue_def_beta_column_x[i],blue_def_beta_column_y[i],color='b' )
for i in range(len(red_def_beta_column_x )):
    red_def_beta_column_x[i],red_def_beta_column_y[i]=sort_dots(red_def_beta_column_x[i],red_def_beta_column_y[i])
    plt.scatter(red_def_beta_column_x[i],red_def_beta_column_y[i],color='r' )


#trans_upper
for i in range(len(blue_trans_upper_column_x )):
    blue_trans_upper_column_x[i],blue_trans_upper_column_y[i]=sort_dots(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i])
    plt.scatter(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i],color='r' )
for i in range(len(red_trans_upper_column_x )):
    red_trans_upper_column_x[i],red_trans_upper_column_y[i]=sort_dots(red_trans_upper_column_x[i],red_trans_upper_column_y[i])
    plt.scatter(red_trans_upper_column_x[i],red_trans_upper_column_y[i],color='b')
#trans_lower
for i in range(len(blue_trans_lower_column_x )):
    blue_trans_lower_column_x[i],blue_trans_lower_column_y[i]=sort_dots(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i])
    plt.scatter(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i],color='b' )
for i in range(len(red_trans_lower_column_x )):
    red_trans_lower_column_x[i],red_trans_lower_column_y[i]=sort_dots(red_trans_lower_column_x[i],red_trans_lower_column_y[i])
    plt.scatter(red_trans_lower_column_x[i],red_trans_lower_column_y[i],color='r')
distance_list_pure_beta = []
distance_list_def_beta = []
distance_list_trans_upper = []
distance_list_trans_lower = []
distance_list_pure_beta.extend(plot_center_to_nearest_reds(all_blue_pure_beta_x, all_blue_pure_beta_y, all_red_pure_beta_x, all_red_pure_beta_y, 'g-', x_scale))
distance_list_def_beta.extend(plot_center_to_nearest_reds(all_blue_def_beta_x, all_blue_def_beta_y, all_red_def_beta_x, all_red_def_beta_y, 'g-', x_scale))
distance_list_trans_upper.extend(plot_bonds_upper(all_blue_trans_upper_x, all_blue_trans_upper_y, all_red_trans_upper_x, all_red_trans_upper_y, 'g-', x_scale))
distance_list_trans_lower.extend(plot_bonds_lower(all_blue_trans_lower_x, all_blue_trans_lower_y, all_red_trans_lower_x, all_red_trans_lower_y, 'g-', x_scale))
plt.show()


In [491]:
i_points_In, i_record_In, p_record_In = sublattice_In.integrate_column_intensity(max_radius=3)
i_points_B, i_record_B, p_record_B = sublattice_B.integrate_column_intensity(max_radius=3)
fig, ax = plt.subplots()

i_record_In.plot(ax=ax,cmap='viridis',scalebar=False,alpha =0.50)
i_record_B.plot(ax=ax,cmap='viridis',scalebar=False,alpha =0.50)
s_crop.plot(ax=ax,cmap='gray',alpha =1)
plt.show()

Calculating Voronoi:   0%|          | 0/16830 [00:00<?, ?it/s]

Integrating:   0%|          | 0/74 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/16830 [00:00<?, ?it/s]

Calculating Voronoi:   0%|          | 0/16830 [00:00<?, ?it/s]

Integrating:   0%|          | 0/103 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/16830 [00:00<?, ?it/s]

In [493]:
masked_data_B = np.ma.masked_invalid(i_record_B.data)
masked_data_In = np.ma.masked_invalid(i_record_In.data)
cmap = plt.cm.viridis  
cmap.set_bad(color='none')
fig, ax = plt.subplots()
global_vmin = min(np.min(masked_data_B), np.min(masked_data_In))
global_vmax = max(np.max(masked_data_B), np.max(masked_data_In))

ax.imshow(s_crop.data, cmap='gray', alpha=1)
ax.imshow(masked_data_B, cmap=cmap,  vmin=global_vmin, vmax=global_vmax,alpha=0.6)  # fully opaque
ax.imshow(masked_data_In, cmap=cmap,vmin=global_vmin, vmax=global_vmax, alpha=0.6)
im = ax.imshow(masked_data_In, cmap=cmap, vmin=global_vmin, vmax=global_vmax, alpha=0.5)
cbar = fig.colorbar(im, ax=ax, orientation='vertical')
cbar.set_label('Intensity')

plt.show()

In [494]:
In_Se_ratio = np.average(i_points_In) / np.average(i_points_B)
std_normalized_In = np.std(i_points_In) / np.average(i_points_In)
std_normalized_B = np.std(i_points_B) / np.average(i_points_B)
data_to_save = np.array([In_Se_ratio, std_normalized_In, std_normalized_B])
np.savetxt('Beta In_Se_ratio, StdDev_In_to_Avg_In, StdDev_B_to_Avg_B.txt',data_to_save, header='In_Se_ratio, StdDev_In/Avg_In, StdDev_B/Avg_B')
np.savetxt('Beta In_intensity.txt',i_points_In)
np.savetxt('Beta B_intensity.txt',i_points_B)

In [495]:
import statistics as st
def stat(distance):
    data = distance
    avg = sum(distance)/len(distance)
    std = st.stdev(data)
    count =len(distance)
    return {'data':data, 'average':avg,'stdev':std,'counts':count}
stats_result1 = stat(distance_list_pure_beta)
stats_str1 = "\n".join(f"{key}: {value}" for key, value in stats_result1.items())
stats_result2 = stat(distance_list_def_beta)
stats_str2 = "\n".join(f"{key}: {value}" for key, value in stats_result2.items())
# Save the string to a file
stats_result3 = stat(distance_list_trans_upper)
stats_str3 = "\n".join(f"{key}: {value}" for key, value in stats_result3.items())
stats_result4 = stat(distance_list_trans_lower)
stats_str4 = "\n".join(f"{key}: {value}" for key, value in stats_result4.items())
# Save the string to a file
file_path1 = 'pure beta.txt'  # Specify your desired file path
with open(file_path1, 'w') as file:
    file.write(stats_str1)
# Save the string to a file
file_path2 = 'defective beta.txt'  # Specify your desired file path
with open(file_path2, 'w') as file:
    file.write(stats_str2)
file_path3 = 'transition upper.txt'  # Specify your desired file path
with open(file_path3, 'w') as file:
    file.write(stats_str3)
# Save the string to a file
file_path4 = 'transition lower.txt'  # Specify your desired file path
with open(file_path4, 'w') as file:
    file.write(stats_str4)

In [521]:
def plot_beta(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >90 and bcx <92.5 and rx >97:
                if bcy > ry and d<20:
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                    distance_list.append(d * x_scale)  # Scale and append the distance
            if bcx >54 and bcx <56 and rx >61 and rx<64:
                if bcy > ry and d<20:
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                    distance_list.append(d * x_scale)  # Scale and append the distance
            if bcx >rx and ry < bcy and d <12:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
            elif bcx <rx and ry > bcy and d <12:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
    return distance_list
def plot_bonds_trans(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >16 and bcx <21 and rx >20 and rx<29:
                if bcy > ry and d<15:
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                    distance_list.append(d * x_scale)  # Scale and append the distance
            elif d <7 and d >2:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
    return distance_list
def plot_bonds_lower(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx>17 and bcx<22 and bcy>ry and d <13 and rx>21:
                plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                #distance_list.append(d * x_scale)  # Scale and append the distance
            elif d <9:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d * x_scale)  # Scale and append the distance
    return distance_list

plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_pure_beta_column_x )):
    blue_pure_beta_column_x[i],blue_pure_beta_column_y[i]=sort_dots(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i])
    plt.scatter(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i],color='b' )
for i in range(len(red_pure_beta_column_x )):
    red_pure_beta_column_x[i],red_pure_beta_column_y[i]=sort_dots(red_pure_beta_column_x[i],red_pure_beta_column_y[i])
    plt.scatter(red_pure_beta_column_x[i],red_pure_beta_column_y[i],color='r' )

for i in range(len(blue_def_beta_column_x )):
    blue_def_beta_column_x[i],blue_def_beta_column_y[i]=sort_dots(blue_def_beta_column_x[i],blue_def_beta_column_y[i])
    plt.scatter(blue_def_beta_column_x[i],blue_def_beta_column_y[i],color='b' )
for i in range(len(red_def_beta_column_x )):
    red_def_beta_column_x[i],red_def_beta_column_y[i]=sort_dots(red_def_beta_column_x[i],red_def_beta_column_y[i])
    plt.scatter(red_def_beta_column_x[i],red_def_beta_column_y[i],color='r' )


#trans_upper
for i in range(len(blue_trans_upper_column_x )):
    blue_trans_upper_column_x[i],blue_trans_upper_column_y[i]=sort_dots(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i])
    plt.scatter(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i],color='b' )
for i in range(len(red_trans_upper_column_x )):
    red_trans_upper_column_x[i],red_trans_upper_column_y[i]=sort_dots(red_trans_upper_column_x[i],red_trans_upper_column_y[i])
    plt.scatter(red_trans_upper_column_x[i],red_trans_upper_column_y[i],color='r')
#trans_lower
for i in range(len(blue_trans_lower_column_x )):
    blue_trans_lower_column_x[i],blue_trans_lower_column_y[i]=sort_dots(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i])
    plt.scatter(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i],color='b' )
for i in range(len(red_trans_lower_column_x )):
    red_trans_lower_column_x[i],red_trans_lower_column_y[i]=sort_dots(red_trans_lower_column_x[i],red_trans_lower_column_y[i])
    plt.scatter(red_trans_lower_column_x[i],red_trans_lower_column_y[i],color='r')
distance_special_beta = []
distance_special_zigzag =[]
distance_special_lower = []
distance_special_beta.extend(plot_beta(all_blue_pure_beta_x, all_blue_pure_beta_y, all_red_pure_beta_x, all_red_pure_beta_y, 'g-', x_scale))
distance_special_beta.extend(plot_beta(all_blue_def_beta_x, all_blue_def_beta_y, all_red_def_beta_x, all_red_def_beta_y, 'g-', x_scale))
distance_special_zigzag.extend(plot_bonds_trans(all_blue_trans_upper_x, all_blue_trans_upper_y, all_red_trans_upper_x, all_red_trans_upper_y, 'g-', x_scale))
distance_special_lower.extend(plot_bonds_lower(all_blue_trans_lower_x, all_blue_trans_lower_y, all_red_trans_lower_x, all_red_trans_lower_y, 'g-', x_scale))
plt.show()

In [522]:
stats1 = stat(distance_special_beta)
stats_1 = "\n".join(f"{key}: {value}" for key, value in stats1.items())
stats_2 = "\n".join(str(i) for i in distance_special_zigzag)
stats_3 = "\n".join(str(i) for i in distance_special_lower)
file1 = 'big bond_beta.txt'  # Specify your desired file path
with open(file1, 'w') as file:
    file.write(stats_1)
# Save the string to a file
file_path2 = 'big bond_zigzag.txt'  # Specify your desired file path
with open(file_path2, 'w') as file:
    file.write(stats_2)
file_path3 = 'big bond_transition_lower.txt'  # Specify your desired file path
with open(file_path3, 'w') as file:
    file.write(stats_3)

In [525]:
distance_4plot = []
def plot_beta_only(blue_center_x, blue_center_y, red_x, red_y, x_scale):
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >90 and bcx <92.5 and rx >97:
                if bcy > ry and d<20:
                    d2 = d * x_scale
                    distance_4plot.append(d2)
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
            if bcx >54 and bcx <56 and rx >61 and rx<64:
                if bcy > ry and d<20:
                    d2 = d * x_scale
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                    distance_4plot.append(d2)
def plot_bonds_trans_only(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >16 and bcx <21 and rx >20 and rx<29:
                if bcy > ry and d<15:
                    d2 = d * x_scale
                    plt.plot([bcx, rx], [bcy, ry], 'w')  # Plot line to each of the two nearest red dots
                    distance_4plot.append(d2)
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_pure_beta_column_x )):
    blue_pure_beta_column_x[i],blue_pure_beta_column_y[i]=sort_dots(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i])
    plt.scatter(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i],color='b' )
for i in range(len(red_pure_beta_column_x )):
    red_pure_beta_column_x[i],red_pure_beta_column_y[i]=sort_dots(red_pure_beta_column_x[i],red_pure_beta_column_y[i])
    plt.scatter(red_pure_beta_column_x[i],red_pure_beta_column_y[i],color='r' )

for i in range(len(blue_def_beta_column_x )):
    blue_def_beta_column_x[i],blue_def_beta_column_y[i]=sort_dots(blue_def_beta_column_x[i],blue_def_beta_column_y[i])
    plt.scatter(blue_def_beta_column_x[i],blue_def_beta_column_y[i],color='b' )
for i in range(len(red_def_beta_column_x )):
    red_def_beta_column_x[i],red_def_beta_column_y[i]=sort_dots(red_def_beta_column_x[i],red_def_beta_column_y[i])
    plt.scatter(red_def_beta_column_x[i],red_def_beta_column_y[i],color='r' )

#trans_upper
for i in range(len(blue_trans_upper_column_x )):
    blue_trans_upper_column_x[i],blue_trans_upper_column_y[i]=sort_dots(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i])
    plt.scatter(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i],color='b' )
for i in range(len(red_trans_upper_column_x )):
    red_trans_upper_column_x[i],red_trans_upper_column_y[i]=sort_dots(red_trans_upper_column_x[i],red_trans_upper_column_y[i])
    plt.scatter(red_trans_upper_column_x[i],red_trans_upper_column_y[i],color='r')
#trans_lower
for i in range(len(blue_trans_lower_column_x )):
    blue_trans_lower_column_x[i],blue_trans_lower_column_y[i]=sort_dots(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i])
    plt.scatter(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i],color='b' )
for i in range(len(red_trans_lower_column_x )):
    red_trans_lower_column_x[i],red_trans_lower_column_y[i]=sort_dots(red_trans_lower_column_x[i],red_trans_lower_column_y[i])
    plt.scatter(red_trans_lower_column_x[i],red_trans_lower_column_y[i],color='r')

plot_beta_only(all_blue_pure_beta_x, all_blue_pure_beta_y, all_red_pure_beta_x, all_red_pure_beta_y, x_scale)
plot_beta_only(all_blue_def_beta_x, all_blue_def_beta_y, all_red_def_beta_x, all_red_def_beta_y, x_scale)
plot_bonds_trans_only(all_blue_trans_upper_x, all_blue_trans_upper_y, all_red_trans_upper_x, all_red_trans_upper_y, 'g-', x_scale)
plot_bonds_trans_only(all_blue_trans_lower_x, all_blue_trans_lower_y, all_red_trans_lower_x, all_red_trans_lower_y, 'g-', x_scale)
plt.show()

In [526]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
norm = mcolors.Normalize(vmin=min(distance_4plot), vmax=max(distance_4plot))
# Step 1: Define the colors for the colormap (white to gray)
colors = ["white","white", 'yellow', 'c',"black","black"]  # You can specify a hex code or RGB tuple for precise color control

# Step 2: Create the colormap
cmap2 = mcolors.LinearSegmentedColormap.from_list("custom_white_gray", colors)

# Step 3: Test the colormap with a simple plot
gradient_x = np.linspace(0, 1, 256)  # Create a gradient from 0 to 1
gradient = np.tile(gradient_x, (1, 1))  # Tile the gradient to make a 2D array
plt.imshow(gradient, aspect='auto', cmap=cmap2)

In [527]:
def plot_beta_only(blue_center_x, blue_center_y, red_x, red_y, x_scale):
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >90 and bcx <92.5 and rx >97:
                if bcy > ry and d<20:
                    d2 = d * x_scale
                    plt.plot([bcx, rx], [bcy, ry], color = cmap2(norm(d2)))  # Plot line to each of the two nearest red dots
            if bcx >54 and bcx <56 and rx >61 and rx<64:
                if bcy > ry and d<20:
                    d2 = d * x_scale
                    color = cmap(norm(d2))
                    plt.plot([bcx, rx], [bcy, ry], color = cmap2(norm(d2)))  # Plot line to each of the two nearest red dots
                    
def plot_bonds_trans_only(blue_center_x, blue_center_y, red_x, red_y,  x_scale):
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >16 and bcx <21 and rx >20 and rx<29:
                if bcy > ry and d<15 and d>4:
                    d2 = d * x_scale
                    color = cmap(norm(d2))
                    plt.plot([bcx, rx], [bcy, ry], color = cmap2(norm(d2)))  # Plot line to each of the two nearest red dots
plt.imshow(s_crop.data, cmap='gray')
for i in range(len(blue_pure_beta_column_x )):
    blue_pure_beta_column_x[i],blue_pure_beta_column_y[i]=sort_dots(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i])
    plt.scatter(blue_pure_beta_column_x[i],blue_pure_beta_column_y[i],color='b' )
for i in range(len(red_pure_beta_column_x )):
    red_pure_beta_column_x[i],red_pure_beta_column_y[i]=sort_dots(red_pure_beta_column_x[i],red_pure_beta_column_y[i])
    plt.scatter(red_pure_beta_column_x[i],red_pure_beta_column_y[i],color='r' )

for i in range(len(blue_def_beta_column_x )):
    blue_def_beta_column_x[i],blue_def_beta_column_y[i]=sort_dots(blue_def_beta_column_x[i],blue_def_beta_column_y[i])
    plt.scatter(blue_def_beta_column_x[i],blue_def_beta_column_y[i],color='b' )
for i in range(len(red_def_beta_column_x )):
    red_def_beta_column_x[i],red_def_beta_column_y[i]=sort_dots(red_def_beta_column_x[i],red_def_beta_column_y[i])
    plt.scatter(red_def_beta_column_x[i],red_def_beta_column_y[i],color='r' )

#trans_upper
for i in range(len(blue_trans_upper_column_x )):
    blue_trans_upper_column_x[i],blue_trans_upper_column_y[i]=sort_dots(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i])
    plt.scatter(blue_trans_upper_column_x[i],blue_trans_upper_column_y[i],color='b' )
for i in range(len(red_trans_upper_column_x )):
    red_trans_upper_column_x[i],red_trans_upper_column_y[i]=sort_dots(red_trans_upper_column_x[i],red_trans_upper_column_y[i])
    plt.scatter(red_trans_upper_column_x[i],red_trans_upper_column_y[i],color='r')
#trans_lower
for i in range(len(blue_trans_lower_column_x )):
    blue_trans_lower_column_x[i],blue_trans_lower_column_y[i]=sort_dots(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i])
    plt.scatter(blue_trans_lower_column_x[i],blue_trans_lower_column_y[i],color='b' )
for i in range(len(red_trans_lower_column_x )):
    red_trans_lower_column_x[i],red_trans_lower_column_y[i]=sort_dots(red_trans_lower_column_x[i],red_trans_lower_column_y[i])
    plt.scatter(red_trans_lower_column_x[i],red_trans_lower_column_y[i],color='r')

plot_beta_only(all_blue_pure_beta_x, all_blue_pure_beta_y, all_red_pure_beta_x, all_red_pure_beta_y,  x_scale)
plot_beta_only(all_blue_def_beta_x, all_blue_def_beta_y, all_red_def_beta_x, all_red_def_beta_y,  x_scale)
plot_bonds_trans_only(all_blue_trans_upper_x, all_blue_trans_upper_y, all_red_trans_upper_x, all_red_trans_upper_y,  x_scale)
plot_bonds_trans_only(all_blue_trans_lower_x, all_blue_trans_lower_y, all_red_trans_lower_x, all_red_trans_lower_y, x_scale)
plt.show()